# Play with Galactic orbits of BHs
author: [Mathieu Renzo](mrenzo@flatironinstitute.org)

In [ ]:
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

import gala as ga

import sys
sys.path.append("../src")
from kicks import integrate_orbits_with_kicks

import socket
if socket.gethostname() == "ccalin010.flatironinstitute.org":
    sys.path.append('/mnt/home/mrenzo/codes/python_stuff/plotFunc/')
    from plotDefaults import * #set_plot_defaults_from_matplotlibrc
elif socket.gethostname() == 't490s':
    sys.path.append('/home/math/Documents/Research/codes/plotFunc/')
    from plotDefaults import * #set_plot_defaults_from_matplotlibrc

# to draw kicks from Maxwellians
from scipy.stats import norm
import random
import scipy.optimize
from scipy.stats import maxwell, uniform
%load_ext autoreload
%autoreload 2

In [ ]:
if socket.gethostname() == "ccalin010.flatironinstitute.org":
    set_plot_defaults_from_matplotlibrc("/mnt/home/mrenzo/codes/python_stuff/plotFunc/")
elif socket.gethostname() == 't490s':
    set_plot_defaults_from_matplotlibrc('/home/math/Documents/Research/codes/plotFunc/')

In [ ]:
# define reference frame
galcen_frame = coord.Galactocentric()

In [ ]:
# define Sun position
sun_w0 = gd.PhaseSpacePosition( pos=[-8.2, 0, 0.02] * u.kpc,
                                vel=galcen_frame.galcen_v_sun )


In [ ]:
c = coord.SkyCoord(ra="17:51:40.2082", dec="-29:53:26.502", unit=(u.hourangle, u.degree))
cosdec = np.cos(c.dec) 

w0s = []

# Tom's RV sample
# rvs = [0,0]

rvs = np.linspace(-100,100,5)
# Mathieu: rv seems not to matter: is this because it is towards the GC?

for rv in rvs: 
    c = coord.SkyCoord(
        ra="17:51:40.2082",
        dec="-29:53:26.502",
        unit=(u.hourangle, u.degree),
        distance=1.58*u.kpc,
        pm_ra_cosdec=-4.36*u.mas/u.yr,# * cosdec,
        pm_dec=3.06*u.mas/u.yr,
        radial_velocity=rv*u.km/u.s
    )

    w0 = gd.PhaseSpacePosition(c.transform_to(galcen_frame).data)
    w0s.append(w0)
    
w0s = gd.combine(w0s)

In [ ]:
# define max time and num timesteps
t_max = 4*u.Gyr
n_steps= 4000

In [ ]:
pot = gp.MilkyWayPotential()
orbits = pot.integrate_orbit(w0s, dt=t_max / n_steps, n_steps=n_steps)
sun_orbit = pot.integrate_orbit(sun_w0, t=orbits.t)

In [ ]:
no_kicks = integrate_orbits_with_kicks(pot, w0s, dt=t_max / n_steps, n_steps=n_steps)

In [ ]:
# ensemble of plots with unperturbed orbits
fig, ax = plt.subplots(figsize=(12, 8))

# plot the sun
_ = sun_orbit.cylindrical.plot(['rho', 'z'], axes=[ax], color='black', zorder=10, alpha=0.5)

xlim = ax.get_xlim()
ylim = ax.get_ylim()

# plot the orbits with kicks
for i in range(no_kicks.shape[1]):
    _ = no_kicks[:, i].cylindrical.plot(['rho', 'z'], axes=[ax],
                                        label=r"$v_{r}="+f"{rvs[i]:.0f}"+r"$", zorder=no_kicks.shape[1]-i,
                                        alpha=0.5, linestyle="-", lw=1)
# _ = no_kicks.cylindrical.plot(['rho', 'z'], axes=[ax], alpha=0.5) #, color='tab:purple', alpha=0.5)


# ax.set_xlim(min(ax.get_xlim()[0], xlim[0]), max(ax.get_xlim()[1], xlim[1]))
# ax.set_ylim(min(ax.get_ylim()[0], ylim[0]), max(ax.get_ylim()[1], ylim[1]))
ax.set_xlim(2.5, 12.5)
ax.set_ylim(-3,3)

ax.plot(np.nan, np.nan, color='k', label=r"$\odot$")
ax.legend(frameon=True, loc="lower left", ncol=2)

In [ ]:
# unkicked eccentricity distribution
n_orbits = 1000
rvs = np.linspace(-100,100, n_orbits)
weights = uniform.rvs(loc=0, size=n_orbits)
# Mathieu: rv seems not to matter: is this because it is towards the GC?
w0s = []
for rv in rvs: 
    c = coord.SkyCoord(
        ra="17:51:40.2082",
        dec="-29:53:26.502",
        unit=(u.hourangle, u.degree),
        distance=1.58*u.kpc,
        pm_ra_cosdec=-4.36*u.mas/u.yr,# * cosdec,
        pm_dec=3.06*u.mas/u.yr,
        radial_velocity=rv*u.km/u.s
    )

    w0 = gd.PhaseSpacePosition(c.transform_to(galcen_frame).data)
    w0s.append(w0)
    
w0s = gd.combine(w0s)

no_kicks = integrate_orbits_with_kicks(pot, w0s, dt=t_max / n_steps, n_steps=n_steps)

ecc = no_kicks.eccentricity().value
fig, ax = plt.subplots(figsize=(12, 8))
ax.hist(ecc, weights=weights)

In [ ]:
def mk_frame(pot, w0s, kick=None, kick_time, t_max, n_steps, sun_orbit=None, no_kicks=None, figname=None, sigma=None):
    fig, ax = plt.subplots(figsize=(12, 8))

    # plot the sun
    if not sun_orbit:
        # calculate the Sun orbit
        pot = gp.MilkyWayPotential()
        orbits = pot.integrate_orbit(w0s, dt=t_max / n_steps, n_steps=n_steps)
        sun_orbit = pot.integrate_orbit(sun_w0, t=orbits.t)
    _ = sun_orbit.cylindrical.plot(['rho', 'z'], axes=[ax], color='red', zorder=10, label="$\odot$")

    # plot with no kicks
    if not no_kicks:
        # calculate the unperturbed orbit
        pot = gp.MilkyWayPotential()
        orbits = pot.integrate_orbit(w0s, dt=t_max / n_steps, n_steps=n_steps)
        no_kicks = integrate_orbits_with_kicks(pot, w0s, dt=t_max / n_steps, n_steps=n_steps)    
    #_ = no_kicks.cylindrical.plot(['rho', 'z'], axes=[ax]) #, color='black', alpha=0.5, zorder=10)
    for i in range(no_kicks.shape[1]):
        _ = no_kicks[:, i].cylindrical.plot(['rho', 'z'], axes=[ax], label=r"$v_{r}="+f"{rvs[i]:.0f}"+r"$", zorder=no_kicks.shape[1]-i, alpha=0.5, linestyle=":")

    # create kicked orbit:
    if kick:
        kicked = integrate_orbits_with_kicks(pot, w0s, kicks=[kick], kick_times=[kick_time], dt=t_max / n_steps, n_steps=n_steps)
        for i in range(kicked.shape[1]):
            _ = kicked[:, i].cylindrical.plot(['rho', 'z'], axes=[ax], label=r"angle",  alpha=0.75)
    elif sigma:
        kicked = integrate_orbits_with_kicks(pot, w0s, kicks=None, kick_times=[kick_time], dt=t_max / n_steps, n_steps=n_steps, maxwell_sigma=sigma)        
        for i in range(kicked.shape[1]):
            _ = kicked[:, i].cylindrical.plot(['rho', 'z'], axes=[ax], label=r"angle",  alpha=0.75)
    else:
        raise ValueError("Either give a list of kick amplitudes, a
        list of tuples for amplitudes, theta and phi, or a sigma for
        maxwellian distribution")    

    ax.set_xlim(2.5, 12.5)# min(ax.get_xlim()[0], xlim[0]), max(ax.get_xlim()[1], xlim[1]))
    ax.set_ylim(-3,3) #min(ax.get_ylim()[0], ylim[0]), max(ax.get_ylim()[1], ylim[1]))
    # ax.set_title(r"$v_{\rm kick}="+f"{kick.value:.0f}"+r"\,\mathrm{km\ s^{-1}}$", fontsize=30)
    # ax.plot(np.nan, np.nan, c='k', label="No kick")

    # ax.plot(np.nan, np.nan, c='m', label=r"$v_{\rm kick}="+f"{kick.value:.0f}"+r"\,\mathrm{km\ s^{-1}}$")
    ax.legend(handlelength=0.5, frameon=True, loc="lower left", ncol=1, fontsize=20)
    if figname:
        plt.savefig('/mnt/home/mrenzo/TMP/kicked_bh/'+str(figname)+'.png')
        plt.close()
    else:
        plt.show()


In [ ]:
# test
n_kicks=1
# The BH has M~7Msun~M_he so single star progenitor would be ~20Msun with lifetime 10Myr
kick_times = [0.01] * u.Gyr
# define maxwellian distribution
sigma = 10 # *u.km/u.s # km/s
kicks = [45*u.km/u.s]*n_kicks

n_kicks  = 2
for i in range(n_kicks):
    mk_frame(pot, w0s, kick_times, t_max, n_steps, sun_orbit=sun_orbit, no_kicks=no_kicks, sigma=10) #, figname=figname)

In [ ]:
fig = plt.figure(figsize=(12,8))
gs = gridspec.GridSpec(2,3)

rho = sun_orbit.cylindrical.pos.rho
phi = sun_orbit.cylindrical.pos.phi
z = sun_orbit.pos.z

ax= fig.add_subplot(gs[0])
ax.plot(rho, phi)
ax.set_xlim(7.5, 10)
ax.set_ylim(-2*np.pi, 2*np.pi)
ax= fig.add_subplot(gs[1])
ax.plot(rho, z)
ax.set_xlim(8, 12)
ax.set_ylim(-0.5, 0.5)
ax= fig.add_subplot(gs[2])
ax.plot(phi, z)
ax.set_xlim(-2*np.pi, 2*np.pi)
ax.set_ylim(-0.5, 0.5)


v_r = sun_orbit.cylindrical.vel


# v_x = vel[0]
# v_y = vel[1]
# v_z = vel[3]

# ax= fig.add_subplot(gs[3])
# ax.plot(v_x, v_y)
# ax= fig.add_subplot(gs[4])
# ax.plot(v_x, v_z)
# ax= fig.add_subplot(gs[5])
# ax.plot(v_y, v_z)


# ax.plot(pos)